In [ ]:
import pandas as pd
import warnings

from sqlalchemy import create_engine
from sqlalchemy.sql import text

warnings.filterwarnings('ignore')

### Data Downloading

In [2]:
banks_df = pd.read_excel('../data/banks.xlsx')
offers_df = pd.read_excel('../data/offers.xlsx')

In [3]:
banks_df.head()

,id,name,rate,online_rate,term,amount_from,amount_to,offer_count,final_rate
0,1,Транскапиталбанк,24.00,0.00,181 день,100000,3000000.0,3,24.00
1,2,Локо-Банк,22.20,0.00,370 дней,100000,5000000.0,1,22.20
2,3,Московский Кредитный Банк,25.00,25.00,95 – 1100 дней,10000,3000000.0,0,25.00
3,4,Газпромбанк,24.00,0.01,от 1 дня,1,1500000.0,0,24.00
4,5,Альфа-Банк,21.05,22.02,92 – 1095 дней,50000,NaN,3,22.02


In [4]:
offers_df.head()

,bank_id,bank_name,rate,online_rate,term,amount_from,amount_to,final_rate
0,1,Транскапиталбанк,23.00,0.0,367 дней,100000.0,3000000.0,23.00
1,1,Транскапиталбанк,22.50,0.0,91 день,100000.0,3000000.0,22.50
2,1,Транскапиталбанк,10.25,0.0,1110 дней,100000.0,3000000.0,10.25
3,2,Локо-Банк,21.60,0.0,182 дня,100000.0,5000000.0,21.60
4,5,Альфа-Банк,19.22,21.0,92 – 1095 дней,10000.0,NaN,21.00


### Подключение к Postgres

In [41]:

# определяем параметры подлчения к postgres
user_name = 'root'
pwd = 'root'
host = 'localhost'
port = 5432
db_name = 'postgres'

# названия таблиц
banks_table_name = 'banks'
offers_table_name = 'offers'

# создаем подключение к postgres
engine = create_engine(f'postgresql://{user_name}:{pwd}@{host}:{port}/{db_name}')

### Создаем Таблицы в БД
Для создание таблицы можно сипользовать несколько способов
- Напрямую вставлять данные
- Используя `pd.io.sql.get_schema`

In [49]:
# способ 1
banks_df.to_sql(name=banks_table_name, con=engine, if_exists='replace')
offers_df.to_sql(name=offers_table_name, con=engine, if_exists='replace')

771

In [50]:
# способ 2
print(pd.io.sql.get_schema(banks_df, name=banks_table_name))

CREATE TABLE "banks" (
"id" INTEGER,
  "name" TEXT,
  "rate" REAL,
  "online_rate" REAL,
  "term" TEXT,
  "amount_from" INTEGER,
  "amount_to" REAL,
  "offer_count" INTEGER,
  "final_rate" REAL
)


In [52]:
# создаем таблицы в БД, если таблицы уже созданые, выведет ошибку, поэтому используем try-except
try:
    with engine.connect() as connection:
        connection.execute(
            # используем text из sqlalchemy, так как он преоразует строку создания таблицы в нужный формат
            # без этого, ругается!
            text(pd.io.sql.get_schema(offers_df, name=offers_table_name, con=engine))
        )
        connection.execute(
            text(pd.io.sql.get_schema(banks_df, name=banks_table_name, con=engine))
        )
except:
    print('Одна из таблиц уже существует! Удали ее!')


Одна из таблиц уже существует! Удали ее!


### Получение Данные из БД

In [69]:
query = """
SELECT *
FROM banks
LIMIT 10
"""

with engine.connect() as connection:
    result = connection.execute(
        text(query)
    )
    result = result.fetchall()

result

[(0, 1, 'Транскапиталбанк', 24.0, 0.0, '181 день', 100000, 3000000.0, 3, 24.0),
 (1, 2, 'Локо-Банк', 22.2, 0.0, '370 дней', 100000, 5000000.0, 1, 22.2),
 (2, 3, 'Московский Кредитный Банк', 25.0, 25.0, '95 – 1100 дней', 10000, 3000000.0, 0, 25.0),
 (3, 4, 'Газпромбанк', 24.0, 0.01, 'от 1 дня', 1, 1500000.0, 0, 24.0),
 (4, 5, 'Альфа-Банк', 21.05, 22.02, '92 – 1095 дней', 50000, None, 3, 22.02),
 (5, 6, 'Т-Банк', 21.0, 24.13, '2 – 24 месяцев', 50000, None, 1, 24.13),
 (6, 7, 'МТС Банк', 21.57, 0.0, '91 – 1096 дней', 10000, None, 0, 21.57),
 (7, 8, 'Газпромбанк', 23.01, 25.0, '181 день', 15000, None, 0, 25.0),
 (8, 9, 'Яндекс Банк', 22.0, 22.0, '91 – 730 дней', 10000, 10000000.0, 0, 22.0),
 (9, 10, 'Россельхозбанк', 23.0, 21.0, '91 – 1095 дней', 5000, None, 0, 23.0)]